# **FAO Data Cleaning Pipeline**
This notebook is designed to perform a comprehensive cleaning and transformation process on the FAOSTAT datasets after they have been downloaded. It will read the raw CSV files from the */kaggle/working/ directory*, perform several cleaning steps, and save the resulting tidy data to a new Clean directory.

## **1. Set Up Paths and Logging**
First, we'll configure the project's directory structure and set up the logging system. On Kaggle, the working directory is /kaggle/working/, so we'll adjust the paths to reflect this.

In [ ]:
import pandas as pd
from pathlib import Path
import logging
from datetime import datetime
import os

# ====================================================================
# === 1. Path Configuration ===
# ====================================================================
# The base path for reading data is the read-only Kaggle input directory.
BASE_DATA_DIR_PATH = Path("/kaggle/input/fao-americas-bulk-data/FAO_Data_Pipeline")
RAW_DIR = BASE_DATA_DIR_PATH / "Raw"

# The path for writing cleaned data is the writable Kaggle working directory.
CLEAN_DIR = Path("/kaggle/working/FAO_Data_Pipeline/Clean")

# The path for logs is also in the writable directory.
LOG_DIR = Path("/kaggle/working/FAO_Data_Pipeline/Logs")

# Create directories for output if they don't exist
CLEAN_DIR.mkdir(parents=True, exist_ok=True)
LOG_DIR.mkdir(parents=True, exist_ok=True)


# ====================================================================
# === 2. Configure Logging ===
# ====================================================================
log_filename = datetime.now().strftime("data_cleaning_log_%Y-%m-%d_%H-%M-%S.log")
log_filepath = LOG_DIR / log_filename

logging.basicConfig(
    filename=str(log_filepath),
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
)


## **2. Define Cleaning Rules and Country Codes**
This cell contains the dictionaries and lists that define the cleaning logic, including the specific columns to keep for each dataset and the FAO area codes for countries in the Americas.

In [ ]:
# ====================================================================
# === 3. Define Columns and Country Codes by Region ===
# ====================================================================
# This dictionary maps each file to the columns that must be kept
dataset_columns = {
    "Consumer_Price_Indices": ["Area Code", "Area", "Months", "Element", "Unit"],
    "Credit_to_Agriculture": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Crops_and_Livestock_Products_Indicators": ["Area Code", "Area", "Indicator", "Unit"],
    "Emissions_Indicators": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Emissions_totals": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Fertilizers_by_Product": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Land_Use": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Macro_Indicators": ["Area Code", "Area", "Element", "Unit"],
    "Pesticide_Use": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Producer_Prices": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Production_Indices": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Value_of_Agricultural_Production": ["Area Code", "Area", "Item", "Element", "Unit"],
    "Value_Shares_by_Industry_and_Primary_Factors": ["Area Code", "Area", "Industry", "Factor", "Element", "Unit"],
}

# FAO area codes for each region of the Americas
caribbean_codes = [
    8, 22, 12, 14, 36, 49, 55, 56, 86, 87, 93, 109, 135, 177, 188, 189, 190, 191, 220, 224, 239, 240, 258, 142, 151
]
central_north_america_codes = [
    23, 48, 60, 89, 95, 138, 157, 166, 33, 231
]
south_america_codes = [
    9, 19, 21, 40, 44, 58, 69, 91, 169, 170, 207, 234, 236
]

# Mapping of area codes to their regions for easy assignment
region_mapping = {}
for code in caribbean_codes:
    region_mapping[code] = "Caribbean"
for code in central_north_america_codes:
    region_mapping[code] = "CentralNorthAmerica"
for code in south_america_codes:
    region_mapping[code] = "SouthAmerica"



## **Implement the Core Cleaning Function** 

This function, clean_fao_data, contains the main logic for reading, transforming, and saving each dataset. It handles various tasks, including reshaping the data, dropping null values, and adding the region column.

In [ ]:
# ====================================================================
# === 4. Data Cleaning Function ===
# ====================================================================
def clean_fao_data(file_path):
    """
    Reads a CSV file, selects the specified columns,
    converts from 'wide' to 'long' format, removes null records,
    and adds a region column before saving.
    """
    file_name = file_path.stem  # Get the file name without extension
    logging.info(f"Processing file: {file_name}")

    if file_name not in dataset_columns:
        logging.warning(f"No column configuration found for {file_name}. Skipping.")
        return

    try:
        try:
            df = pd.read_csv(file_path, encoding="utf-8")
        except UnicodeDecodeError:
            logging.warning(
                f"Failed to read {file_name} with utf-8. Trying with latin1 encoding."
            )
            df = pd.read_csv(file_path, encoding="latin1")

        keep_columns = dataset_columns[file_name]
        year_columns = [col for col in df.columns if col.startswith("Y")]
        existing_keep_columns = [col for col in keep_columns if col in df.columns]
        missing_columns = [col for col in keep_columns if col not in df.columns]

        if missing_columns:
            logging.warning(
                f"Missing columns for {file_name}: {missing_columns}. Skipping this file."
            )
            return

        id_vars = existing_keep_columns
        value_vars = year_columns

        if not value_vars:
            logging.warning(
                f"No year columns found for {file_name}. Skipping this file."
            )
            return

        df_melted = pd.melt(
            df,
            id_vars=id_vars,
            value_vars=value_vars,
            var_name="Year",
            value_name="Value",
        )

        initial_rows = len(df_melted)
        df_melted.dropna(subset=["Value"], inplace=True)
        dropped_rows_nulls = initial_rows - len(df_melted)

        logging.info(
            f"Dropped {dropped_rows_nulls} records with null values in 'Value' column from {file_name}"
        )
        print(
            f"Removed {dropped_rows_nulls} empty 'Value' records from {file_name}.csv"
        )

        df_melted["Year"] = df_melted["Year"].str.replace("Y", "").astype(int)

        all_americas_codes = (
            caribbean_codes + central_north_america_codes + south_america_codes
        )
        initial_rows_before_filter = len(df_melted)
        df_filtered = df_melted[df_melted["Area Code"].isin(all_americas_codes)]
        dropped_rows_filter = initial_rows_before_filter - len(df_filtered)

        logging.info(
            f"Dropped {dropped_rows_filter} records not belonging to Americas from {file_name}"
        )
        print(
            f"Removed {dropped_rows_filter} records not in Americas from {file_name}.csv"
        )

        df_filtered["Region"] = df_filtered["Area Code"].map(region_mapping)
        df_filtered.rename(columns={"Area": "Country"}, inplace=True)

        cleaned_file_path = CLEAN_DIR / f"{file_name}_cleaned.csv"
        df_filtered.to_csv(cleaned_file_path, index=False)

        logging.info(f"Successfully cleaned and saved: {cleaned_file_path}")
        print(f"Cleaned {file_name}.csv and saved to {cleaned_file_path}")

    except Exception as e:
        logging.error(f"Error cleaning {file_name}: {e}")
        print(f"Error cleaning {file_name}: {e}")

## **Run the Data Cleaning Pipeline**
This is the final step. Executing this cell will call the main function, run_fao_cleaning_pipeline, which will iterate through the files in the raw data directory and apply the cleaning function to each one.

In [ ]:
# ====================================================================
# === 5. Run the Cleaning Pipeline ===
# ====================================================================
def run_fao_cleaning_pipeline():
    """
    Main function to run the cleaning pipeline on all
    files in the 'Raw' folder.
    """
    logging.info("Starting FAO data cleaning pipeline")

    raw_files = list(RAW_DIR.glob("*.csv"))

    if not raw_files:
        logging.warning(
            "No CSV files found in the 'Raw' directory. Please ensure the data has been added to this notebook."
        )
        print(
            "No CSV files found in the 'Raw' directory. Please ensure the data has been added to this notebook."
        )
        return

    for file in raw_files:
        clean_fao_data(file)

    logging.info("FAO data cleaning pipeline finished")
    print("All files processed.")


if __name__ == "__main__":
    run_fao_cleaning_pipeline()